# Strands AgentでModel Context Protocol（MCP）をツールとして使用する

## 概要
[Model Context Protocol（MCP）](https://modelcontextprotocol.io/introduction)は、アプリケーションが大規模言語モデル（LLM）にコンテキストを提供する方法を標準化するオープンプロトコルです。Strands AI SDKはMCPと統合し、外部ツールやサービスを通じてエージェントの機能を拡張します。

MCPは、追加のツールを提供するMCPサーバーとエージェント間の通信を可能にします。Strands Agent SDKには、MCPサーバーへの接続とそのツールの使用のための組み込みサポートが含まれています。

この例では、Strands AgentでMCPツールを使用する方法を示します。[AWS Documentation MCPサーバー](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/)を使用します。これは、AWSドキュメントへのアクセス、コンテンツの検索、推奨事項の取得を行うツールを提供します。このMCPサーバーには3つの主な機能があります：

- **ドキュメントの読み取り**: AWSドキュメントページを取得してMarkdown形式に変換
- **ドキュメントの検索**: 公式検索APIを使用してAWSドキュメントを検索
- **推奨事項**: AWSドキュメントページのコンテンツ推奨事項を取得



## エージェントの詳細
<div style="float: left; margin-right: 20px;">
    
|機能                |説明                                        |
|--------------------|-------------------------------------------|
|使用する機能         |MCPツール                                   |
|エージェント構造     |シングルエージェントアーキテクチャ              |

</div>

## アーキテクチャ

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## 主な機能
* **シングルエージェントアーキテクチャ**: この例では、MCPツールとやり取りする単一のエージェントを作成します
* **MCPツール**: エージェントとのMCPツールの統合

## セットアップと前提条件

### 前提条件
* Python 3.10以上
* AWSアカウント
* Amazon BedrockでAnthropic Claude 3.7が有効化されていること

それでは、Strands Agentエージェントに必要なパッケージをインストールしましょう

In [ ]:
# installing pre-requisites
!pip install -r requirements.txt

### 依存パッケージのインポート

それでは、依存パッケージをインポートしましょう

In [ ]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient

### stdioトランスポートを使用してMCPサーバーに接続

MCPの[トランスポート](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports)は、クライアントとサーバー間の通信の基盤を提供します。メッセージの送受信方法の基礎的なメカニズムを処理します。現時点では、MCPに組み込まれている3つの標準トランスポート実装があります：

- **標準入出力（stdio）**: 標準入出力ストリームを通じた通信を可能にします。ローカル統合やコマンドラインツールに特に便利です
- **Streamable HTTP**: これは以前のプロトコルバージョンのHTTP+SSEトランスポートを置き換えます。Streamable HTTPトランスポートでは、サーバーは複数のクライアント接続を処理できる独立したプロセスとして動作します。このトランスポートはHTTP POSTおよびGETリクエストを使用します。サーバーはオプションでServer-Sent Events（SSE）を使用して複数のサーバーメッセージをストリーミングできます。これにより、基本的なMCPサーバーだけでなく、ストリーミングやサーバーからクライアントへの通知やリクエストをサポートするより機能豊富なサーバーが可能になります。
- **SSE**: Server-Sent Eventsトランスポートを使用するHTTPベースのMCPサーバー用のレガシートランスポート  

全体として、コマンドラインツールの構築、ローカル統合の実装、シェルスクリプトの操作にはstdioを使用する必要があります。AIエージェントがツールやサービスと通信するための柔軟で効率的な方法が必要な場合、特にステートレス通信を扱う場合やリソース使用量を最小限に抑えることが重要な場合は、Streamable HTTPトランスポートを使用する必要があります。

特定のニーズに合わせて**カスタムトランスポート**実装を使用することもできます。


それでは、stdioトランスポートを使用してMCPサーバーに接続しましょう。まず、`MCPClient`クラスを使用して[AWS Documentation MCPサーバー](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/)に接続します。このサーバーは、AWSドキュメントへのアクセス、コンテンツの検索、推奨事項の取得を行うツールを提供します。

In [ ]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### エージェント設定のセットアップと呼び出し

次に、先ほど作成した`stdio_mcp_client`オブジェクトからのツールを使用してエージェント設定を行います。そのためには、MCPサーバーで利用可能なツールをリストする必要があります。そのために`list_tools_sync`メソッドを使用できます。

その後、エージェントに質問します。

In [ ]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

### Streamable HTTPを使用してMCPサーバーに接続

それでは、Streamable HTTPトランスポートを使用してMCPサーバーに接続しましょう。まず、Streamable HTTPトランスポートを使用してシンプルなMCPサーバーを起動しましょう。

この例では、独自のMCPサーバーを作成します。アーキテクチャは以下のようになります

<div style="text-align:center">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [ ]:
# Create an MCP server
mcp = FastMCP("Calculator Server")

# Define a tool


@mcp.tool(description="Calculator tool which performs calculations")
def calculator(x: int, y: int) -> int:
    return x + y


@mcp.tool(description="This is a long running tool")
def long_running_tool(name: str) -> str:
    time.sleep(25)
    return f"Hello {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

それでは、`streamable-http`サーバーでスレッドを開始しましょう

In [ ]:
thread = threading.Thread(target=main)
thread.start()

#### Streamable HTTPクライアントとエージェントの統合

それでは、`streamablehttp_client`を使用してこのサーバーをシンプルなエージェントと統合しましょう。

In [ ]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### エージェント設定のセットアップと呼び出し

次に、先ほど作成した`streamable_http_mcp_client`オブジェクトからのツールを使用してエージェント設定を行います。そのためには、MCPサーバーで利用可能なツールをリストする必要があります。そのために`list_tools_sync`メソッドを使用できます。

その後、エージェントに質問します。

In [ ]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = str(agent("What is 2 + 2?"))

### 直接ツール呼び出し

ツールは通常、ユーザーのリクエストに基づいてエージェントによって呼び出されますが、MCPツールを直接呼び出すこともできます。これは、複数のツールをまとめてオーケストレーションするワークフローシナリオで便利です。

In [ ]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # direct tool invocation
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    # Process the result
    print(f"Calculation result: {result['content'][0]['text']}")

MCPサーバーツールを呼び出す際に、実行時間が長くなりすぎないように`read_timeout_seconds`をオプションで指定することもできます

In [ ]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "Amazon"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"Tool execution failed: {result['content'][0]['text']}")
        else:
            print(f"Tool execution succeeded: {result['content'][0]['text']}")
    except Exception as e:
        print(f"Tool call timed out or failed: {str(e)}")

### 複数のMCPサーバーとのやり取り

Strands Agentsでは、同じエージェントを使用して複数のMCPサーバーとやり取りし、並行して使用できるツールの最大数（`max_parallel_tools`）などのツール設定を構成することもできます。この設定を示すために新しいエージェントを作成しましょう：

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

このエージェントでは、再びAWS Documentation MCPサーバーを使用し、AWS Cloud Development Kit（CDK）のベストプラクティス、Infrastructure as Codeパターン、CDK Nagによるセキュリティコンプライアンスを支援する[AWS CDK MCPサーバー](https://awslabs.github.io/mcp/servers/cdk-mcp-server/)も使用します。

まず、stdioトランスポートを使用して2つのMCPサーバーに接続しましょう

In [ ]:
# Connect to an MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Connect to an MCP server using stdio transport
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

#### MCPサーバーを使用したエージェントの作成

次に、両方のMCPサーバーからのツールを使用してエージェントを作成します

In [ ]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent(
        "What is Amazon Bedrock pricing model. Be concise. Also what are the best practices related to CDK?"
    )

### おめでとうございます！

このノートブックでは、Strands Agentを使用してMCPサーバーに接続する方法と、2つのMCPトランスポートプロトコル（stdioとStreamable HTTP）について学びました。また、同じエージェントに複数のMCPサーバーを接続する方法も学びました。次は、エージェントでさまざまなモデルを使用する方法を見てみましょう